## Script convert google sheet to obo files (use Pronto)
### TODO
1. To decide the correct identifier of terms (term id). Should it be an url or just a text.
2. To add xrefs of each format.

In [1]:
# load libraries
import pronto 
import pandas as pd

In [2]:
# read xlsx or google sheet
url = "https://docs.google.com/spreadsheets/d/1oS4_GcaFSBXkX4poZgN7mIvfjwWHJBb7ZBjmhmUOkns/export?format=csv&gid=0"
local = "raw_data_format_list.csv"                          
name_list = pd.read_csv(url)

In [3]:
# check the columns
name_list.columns

Index(['extension_ident', 'extension_1', 'format_name', 'type', 'description',
       'rawness', 'semantic_qualitie', 'omics', 'openess', 'EP1',
       'EDAM_ontology', 'other_ontology', 'users_base', 'metadata',
       'reusability', 'achievability', 'structural_transparency '],
      dtype='object')

In [4]:
# read the metadata
terms = pronto.Ontology("ontology/terms_metadata.obo") # todo add metadata by code
class_dict = {'genomics':'ENA','transcriptomics':'GEO','proteomics':'PRIDE', 'metabolomics':'Metabolights'  }

# adding classes
terms.create_term("format")
terms.create_term("ENA")
terms.create_term("GEO")
terms.create_term("PRIDE")
terms.create_term("Metabolights")
# terms.create_term("")




Term('Metabolights')

In [5]:
# loop through the list and create terms
for index, row in name_list.iterrows():
    if row['EP1'] == 'YES': # check if it is accepted in the end point repositories alone or it must be submitted with other files
        new_term = terms.create_term("terms_{}_{}".format(row['extension_ident'], index)) # Term id
        new_term.name = "{} file format".format(row['format_name']) # Term name
        new_term.definition = pronto.Definition("{} file format".format(row['description']), xrefs={pronto.Xref('{}'.format(row["EDAM_ontology"]))})
        new_term.superclasses().add(terms["format"])
        new_term.superclasses().add(terms["{}".format(class_dict[row['omics']])])

# save the obo file
with open("formats_2_end-point_repo.obo", "wb")as f:
    terms.dump(f, format="obo")